### Homework 8

이번 과제에서는 이분산 모형의 추정과 리스크 관리와 관련된 고급 통계 기법들을 실습해보도록 한다

#### 1. Data Import

yfinance library를 이용하여 2020년 1월 1일부터 2023년 7월 31일까지의 IBM 기업의 주가를 import 하여 log defference 형태로 수정하라

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

#### 2. GARCH models

**2.1 GARCH model**

$\mathrm{GARCH}$ 모형은 다음과 같이 추정된다

$$r_t = \mu + a_t$$
$$a_t = \sigma_t \epsilon_t$$
$$\sigma_t^2 = \alpha_0 + \sum_{i=1}^m \alpha_i a_{t-1}^2 + \sum_{j=1}^s \beta_j \sigma_{t-j}^2$$

IBM 수익률 벡터를 활용하여 적절한 order를 선택해 $\mathrm{GARCH}$ model을 추정하고, 추정된 변동성을 plotting하여라

In [ ]:
import arch

**2.2 Exponential GARCH model**

$\mathrm{GARCH}$모형이 금융 시계열에 쓰였을 때 나타나는 단점을 최소화하기 위해서 exponential $\mathrm{GARCH(EGARCH)}$모형을 제안하였다. 좋은 뉴스와 나쁜 뉴스 즉, 음의 충격과 양의 충격에 비대칭적 효과를 고려하여 weight를 조정하였다

$\mathrm{EGARCH}$ 모형은 다음과 같이 나타낼 수 있다

$$r_t = \mu + a_t$$
$$a_t = \sigma_t \epsilon_t$$
$$\ln(\sigma_t^2) = \alpha_0 + \frac{1 +\beta_1 B + \cdots + \beta_{s-1}B^{s-1}}{1 - \alpha_1B - \cdots - \alpha_m B^m}g(\epsilon_{t-1})$$

$$g(\epsilon_t) = \begin{cases}
(\theta + \gamma) \epsilon_t - \gamma E(|\epsilon_t|) ~~~~~ & \mbox{if } \epsilon_t \geq 0\\
(\theta - \gamma) \epsilon_t - \gamma E(|\epsilon_t|) ~~~~~ & \mbox{if } \epsilon_t < 0
\end{cases}$$

한편, python에서는 arch library의 arch_model 함수에서 parameter `vol = EGARCH`로 지정하여 추정할 수 있다

IBM 수익률 벡터에 대해서 $\mathrm{EGARCH}(1,1)$모형을 추정하라. 추정된 변동성에 대해 적절한 방법으로 plotting하여라. **2.1번**의 추정된 결과와 어떤 차이가 있는가?. IBM 주가에 대해서 $\mathrm{EGARCH}(1,1)$ 모형은 유의한가?

#### 3. Value at Risk

재무회계에 있어서 위험은 크게 시장위험, 유동성위험, 신용위험, 운영위험으로 나뉘며, 이중 시장위험을 측정하는데 보편적으로 사용하는 모형은 Value at Risk$\mathrm{(VaR)}$이다. $\mathrm{VaR}$의 측정방법은 다음 네 가지로 분류할 수 있다
1. Econometrics Approach
2. Risk Metrics
3. Quantile Estimation
4. Extreme Value Approach

Econometrics Approach는 $\mathrm{GARCH}$등 변동성 추정 방법을 통해 구하는 방법이며, 다음과 같이 추정한다

$$\mathrm{VaR} = \sigma_t \times z_a$$

여기서 $\sigma_t$는 $\mathrm{GARCH}$모형을 통해 추정된 변동성이며, $z_a$는 한쪽꼬리검정의 신뢰상수를 의미한다. 예를들어, 95%의 신뢰상수는 1.645이며, 99%의 신뢰상수는 2.33이다

Value at Risk를 추정하는 함수를 작성하고 이를 이용해 **Example 2**에서 추정한 두 가지 종류의 $\mathrm{GARCH}$모형에 대해 신뢰수준 95%에서의 일별 $\mathrm{VaR}$을 추정하라. 세 모형의 $\mathrm{VaR}$ 추정값에는 어떤 차이가 있는가?

In [ ]:
def value_at_risk(model : pd.Series, z : float = 2.33) -> pd.Series :
    """
    :param model: GARCH모형 추정시 model.conditional_volatility의 값을 넣으시면 됩니다
    :param z: 신뢰상수를 나타냅니다
    :return: Value ar Risk를 최종적으로 출력하는 함수를 작성하시면 됩니다
    """
    pass

#### 4. Exponential Weighted Moving Average Volatility

변동성을 추정하는 방법은 여러가지가 있지만 그중 지수가중이동평균 변동성(EWMA Volatility)는 계산이 편하고 빠르다는 장점 덕분에 근사값으로 널리 사용되고 있다. 또한, 동적으로 변동성을 추정할 수 있다는 장점도 가지고 있다

EWMA 변동성은 다음과 같이 계산된다

$$\mathrm{EWMA}_t = \lambda r_t + (1-\lambda)\mathrm{EWMA}_{t-1}$$

아래 함수는 일별 변동성을 가중이동평균하여 구하는 함수이다. `span`은 가중이동평균할 기간을 의미하며, 보통 10일을 기준으로 한다

IBM 수익률 시계열에 대하여 10일 가중이동평균 변동성을 구하고 적절한 방법으로 시각화하라. **2.1번**에서 추정한 $\mathrm{GARCH}$ 변동성과 비교하라. EWMA 변동성은 $\mathrm{GARCH}$의 추정치 대신 사용 가능한 지표인가?

In [ ]:
def ewmaVolatility(close : pd.Series, span : int = 10) -> pd.DataFrame :
    df0 = close.index.searchsorted(close.index - pd.Timedelta(days=1))
    df0 = df0[df0 > 0]
    df0 = (pd.Series(close.index[df0 - 1],
                     index=close.index[close.shape[0] - df0.shape[0]:]))
    try:
        df0 = close.loc[df0.index] / close.loc[df0.values].values - 1  
    except Exception as e:
        print(f'error: {e}\nplease confirm no duplicate indices')
    df0 = df0.ewm(span = span).std().rename('dailyVol')
    return df0